In [30]:
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob

In [56]:
!python -m pip install --upgrade pip
!pip install supervision==0.3.0
!pip install transformers
!pip install pytorch-lightning
!pip install timm
!pip install cython
!pip install pycocotools
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [17]:
from google.colab import drive
drive.mount('/content/drive')
# !cp -r ./drive/MyDrive/data/ ./data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Utils functions

In [32]:
def find_relaxed_match_in_array(array, element):
    for obj in array:
        if element in obj:
            return obj

def file_has_data(file):
    with open(file, 'r') as read_obj:
        first_char = read_obj.read(1)
    if first_char.isnumeric():
        return True
    else:
        return False

def bbox_transformer(input_coords, img_size, output_type="cv2"):
    """
    Bounding box converter
    ----------
    input_coords : list
    [x, y, width, height], where `x`, `y` center coords, `w` `h` width, height
    of bounind box. All values in percent of original image resolution
    img_size: tuple
    (width, height), width image in px, height image in px.
    output_type : {'cv2', 'matplotlib'}, default: 'cv2'
    If ``'cv2'``, returns coordinates for cv2.rectangle. If
    ``'matplotlib'`` returns coordinates for matplotlib.patches.Rectangle.
    ----------
    """
    x, y, w, h = input_coords
    width, height = img_size
    if output_type == "cv2":
        x1, y1, x2, y2 = [int(x*width - w*width /2), int(y*height  - h*height/2), int(x1 + w*width), int(y1 + h*height)]
    elif output_type == "matplotlib":
        x1, y1, x2, y2 = [int(x*width - w*width/2), int(y*height- h*height/2), w*width, h*height]
    else:
        raise TypeError("Incorrect output input {output_type}")
    return x1, y1, x2, y2


# View images

In [51]:
DATA_PATH = Path("./data")
IMG_DIR = DATA_PATH / "images"
LABELS_DIR = DATA_PATH / "labels"

IMG_PART = [i for i in IMG_DIR.iterdir()][0]
LABELS_PART= [i for i in LABELS_DIR.iterdir()][0]

images_files = glob.glob(str(IMG_PART)+ "/*")
labels_files = glob.glob(str(LABELS_PART)+ "/*")

In [ ]:
for img_name in images_files[:10]:
    img_file = Path(img_name)
    label_file_matched = find_relaxed_match_in_array(map(str, labels_files), img_file.stem)

    if file_has_data(label_file_matched):
        boxes = np.loadtxt(label_file_matched, delimiter=' ')
        if boxes.ndim == 1:
            boxes = boxes.reshape((1, -1))
        img = Image.open(img_file)
        height, width = img.size
        fig, ax = plt.subplots()
        ax.imshow(img)
        print(f"PEOPLE NUMBER IS {len(boxes)}| {boxes.shape}")
        for box in boxes:
            _, x, y, w, h = box
            x1, y1, x2, y2 = bbox_transformer([x, y, w, h], (height, width),"matplotlib")
            rect = patches.Rectangle((x1,y1), x2, y2, linewidth=0.5, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()


# build COCO dataset

# DETR


In [61]:
import os
import supervision
import transformers
import torchvision
import pytorch_lightning

print(f"{pytorch_lightning.__version__ =}\n{supervision.__version__=}\n{transformers.__version__=}")

pytorch_lightning.__version__ ='2.4.0'
supervision.__version__='0.3.0'
transformers.__version__='4.44.2'


# COCO data loaders

Only one class = ['body']